# Trabalho em Grupo I

```
Disciplina: Inteligência artificial 6º semestre
IFSP campus Campinas
Alunos: 
- Paulo Henrique Belucci - CP3001369
- Silvio da Col de Brito - CP300130X
Prof.: Dr. Samuel Martins
```

## Análise exploratória - Pandemia do Corona vírus no Brasil

---

### Descrição e motivação

A pandemia do Corona vírus teve início ao final do ano 2019 e, em pouco mais de um ano, é responsável por mais de 1,4 milhão de mortes em todo o mundo. Seus sintomas são parecidos com os sintomas da gripe, com destaque para a síndrome respiratória. Embora sua taxa de letalidade não seja tão alta, seu sintomas somados a um grande número de infectados tem se tornado um gravíssimo problema de saúde pública em todos os países, devido a sobrecarga de leitos de Unidade de Tratamento Intensivo (UTI) por aqueles casos se se agravam.

Muitos estudos e pesquisas tem sido desenvolvidos em pro de mapear a genética do vírus, entender as formas de contágio, testar tratamentos para pacientes infectados e, destaca-se, desenvolver uma vacina para imunizar a população interrompendo assim o ciclo de contágio.

Esta análise exploratória visa entender alguns aspectos da pandemia no Brasil, tais informações servem de insumo para elaboração de estratégias eficazes no combate ao avança da doença.



### Descrição da base de dados

sdfsdfsdfsdfs

In [ ]:
import pandas as pd

In [21]:
covid = pd.read_csv('HIST_PAINEL_COVIDBR_02jan2021.csv', delimiter=';')
covid.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-25,9,210147125.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
1,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-26,9,210147125.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN
2,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-27,9,210147125.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN
3,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-28,9,210147125.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN
4,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-29,9,210147125.0,2.0,1.0,0.0,0.0,NaN,NaN,NaN
